In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [63]:
trans = pd.Series([pd.to_datetime('2019-0{}-{}'.format(a,x)) 
                   for a,x in zip(np.random.choice([8,9],size=100),np.random.randint(1,31,size=100))])

In [64]:
df = trans.to_frame('date')
df['value'] = 1000*np.random.random(100)
df['id'] = np.random.randint(11,size=100)
df['transac_desc'] = np.random.choice(['A','B','C','D'],size=100)

In [115]:
df['freq_transac']=df.groupby(['id','transac_desc'])['date'].transform(lambda x : (x - min(x)).dt.days)

In [114]:
df.head()

,date,value,id,transac_desc,freq_transac,interval_transac,day
0,2019-09-29,931.492243,3,D,56,NaN,29
1,2019-08-02,100.185976,8,D,0,NaN,2
2,2019-08-10,579.797091,1,A,0,NaN,10
3,2019-08-07,15.835632,4,A,0,NaN,7
4,2019-08-11,494.701348,5,C,9,NaN,11


In [112]:
def teste(x):
    x = x.sort_values()
    if x.size > 1:
        x = x - x.shift(1)
    return x

In [113]:
df['interval_transac']=df.groupby(['id','transac_desc'])['freq_transac'].transform(teste)

In [106]:
df['day'] = df['date'].apply(lambda x: x.day)

In [166]:
def count_avg(x):
    return x.size/3

In [167]:
final= df.groupby(['id','transac_desc']).agg({'interval_transac':['mean','std'],'day':['min','mean','max','median','std',count_avg,lambda x: max(x)-min(x)],'value':['mean','std']})

In [168]:
final.columns

MultiIndex(levels=[['interval_transac', 'day', 'value'], ['<lambda>', 'count_avg', 'max', 'mean', 'median', 'min', 'std']],
           codes=[[0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2], [3, 6, 5, 3, 2, 4, 6, 1, 0, 3, 6]])

In [169]:
final.columns.get_level_values(0)

Index(['interval_transac', 'interval_transac', 'day', 'day', 'day', 'day',
       'day', 'day', 'day', 'value', 'value'],
      dtype='object')

In [170]:
final.columns = ['_'.join(col).strip() for col in final.columns.values]

In [171]:
final.query('interval_transac_std<3')

interval_transac_mean  interval_transac_std  day_min  \
id transac_desc                                                         
8  C                               6.5              0.707107        2   
   D                               5.0              1.414214        2   
10 D                               2.5              0.707107        4   

                 day_mean  day_max  day_median   day_std  day_count_avg  \
id transac_desc                                                           
8  C             8.666667       15         9.0  6.506407            1.0   
   D             7.333333       12         8.0  5.033223            1.0   
10 D             6.333333        9         6.0  2.516611            1.0   

                 day_<lambda>  value_mean   value_std  
id transac_desc                                        
8  C                       13  378.533857  241.910741  
   D                       10  247.645717  207.301332  
10 D                        5  380.776172  323.363877

In [163]:
np.unique(final.index.get_level_values(0)).shape[0]

11